In [1]:
import pandas as pd
import numpy as np

In [2]:

df=pd.read_csv('BTC-USD.csv',sep=',')
df.head()



,Date,Open,High,Low,Close,Adj Close,Volume
0,13/12/2016,780.646973,788.460022,777.961975,780.556030,780.556030,81645600.0
1,14/12/2016,780.005005,782.033997,776.838989,781.481018,781.481018,75979000.0
2,15/12/2016,780.070007,781.434998,777.802002,778.088013,778.088013,81580096.0
3,16/12/2016,778.963013,785.031982,778.963013,784.906982,784.906982,83608200.0
4,17/12/2016,785.166016,792.508972,784.864014,790.828979,790.828979,78989800.0


In [3]:
time=df[["Date"]].to_numpy()
time.shape
time=list(i for i in range (1827))
time=np.array(time)

print(time[1:5])

close_price=df[["Close"]].to_numpy()
close_price.shape


[1 2 3 4]


(1827, 1)

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train,Y_test=train_test_split(time,close_price,test_size=0.3, random_state=42)

In [5]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1278,)
(1278, 1)
(549,)
(549, 1)


# Preprocessing

In [6]:
Ytrain_mean=Y_train.mean()
Ytest_mean=Y_test.mean()

print(Ytrain_mean)
print(Ytest_mean)


nan
nan


In [7]:
from sklearn import preprocessing
scaler=preprocessing.StandardScaler().fit(Y_train)
print(scaler.mean_)
print(scaler.scale_)
Y_scaled=scaler.transform(Y_train)
Y_scaled

[14802.00139011]
[16679.8263437]


array([[ 0.53843654],
       [-0.44309118],
       [-0.63217573],
       ...,
       [-0.56371307],
       [ 0.19526   ],
       [-0.39915326]])